# 2 model

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def visual(img, ground_truth, prediction, gpu = 0, flag_torch = 0):
  if gpu:
    img = img.cpu().detach()
    ground_truth = ground_truth.cpu().detach()
    prediction = prediction.cpu().detach()

  if flag_torch:
    img = img.numpy()
    ground_truth = ground_truth.numpy()
    prediction = prediction.numpy()

  if flag_torch == 2:
    img = np.transpose(img[:, :, :, :, :], [0, 4, 2, 3, 1]).squeeze()

  #correct image structure
  #use last 5 images so we can easily compare between batch sizes
  img = np.transpose(img[-5:, :, :, :], [0, 2, 3, 1])
  ground_truth = ground_truth[-5:, :, :]
  prediction = prediction[-5:, :, :]

  for i in range(5):
      ax = plt.subplot(3, 5, i + 1)
      ax.imshow(img[i])
      ax.axis("off")
      ax = plt.subplot(3, 5, i + 1 + 5)
      ax.imshow(ground_truth[i])
      ax.axis("off")
      ax = plt.subplot(3, 5, i + 1 + 10)
      ax.imshow(prediction[i])
      ax.axis("off")
  plt.show()

def train(args):
    Net = globals()[args.model]
    #model = Net(args.kernel, args.num_filters)
    model = Net(output_channels = 1)
    train_loader = torch.utils.data.DataLoader(args.data, batch_size=args.batch_size, shuffle=True)

    # Save directory
    save_dir = "outputs/" + args.experiment_name

    # LOSS FUNCTION

    criterion1 = nn.BCELoss()
    criterion2 = nn.MSELoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learn_rate)


    # Create the outputs folder if not created already
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print("Beginning training ...")
    if args.gpu:
        model.cuda()
    start_time = time.time()

    train_losses = []
    valid_losses = []
    valid_accs = []
    epochs = []

    for epoch in range(args.epochs):
        # Train the Model
        model.train()  # Change model to 'train' mode
        losses = []
        for imgs, density, counts in iter(train_loader):
            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = model(imgs.cuda())
            outputs = outputs.squeeze(1)

            loss = criterion2(outputs, density.cuda())


            loss.backward()
            optimizer.step()
            losses.append(loss.data.item())
        train_losses.append(sum(losses))
        epochs.append(epoch)
        print(epoch, train_losses[-1])

        if args.visualize and (epoch+1) % args.visualize_epochs == 0:
            visual(imgs, density, outputs, args.gpu, 1)
    if args.plot:
        # plotting
        plt.title("Training Curve")
        plt.plot(epochs[10:], train_losses[10:], label="Train")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.show()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    torch.save(model.state_dict(), os.path.join(save_dir, "final"))
    return model

## define model

In [ ]:
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ASPP, self).__init__()
        self.conv_1x1_1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv_3x3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv_3x3_2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.conv_3x3_3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=18, dilation=18)
        self.conv_1x1_2 = nn.Conv2d(out_channels * 4, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.conv_1x1_1(x)
        x2 = self.conv_3x3_1(x)
        x3 = self.conv_3x3_2(x)
        x4 = self.conv_3x3_3(x)
        x = torch.cat((x1, x2, x3, x4), dim=1)
        x = self.conv_1x1_2(x)
        return x

class VGG16Modified2(nn.Module):
    def __init__(self, output_channels=1):
        super(VGG16Modified2, self).__init__()
        self.features = models.vgg16(pretrained=True).features
        for param in self.features.parameters():
            param.requires_grad = False

        self.aspp = ASPP(512, 256)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(16, output_channels, kernel_size=2, stride=2),
        )


    def forward(self, x):
        x = self.features(x)
        x = self.aspp(x)
        x = self.decoder(x)


        return x.squeeze()


In [ ]:
# test output size
model = VGG16Modified2(output_channels=1)

# dummy input tensor
dummy_input = torch.randn(1, 3, 224, 224)

with torch.no_grad():
    dummy_output = model(dummy_input)

print("Output size:", dummy_output.size())

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 106MB/s]


Output size: torch.Size([224, 224])


### epoch = 600

In [ ]:
model = VGG16Modified2()
state = torch.load("/content/drive/MyDrive/UT/IDL/project/my_VGG16_mseloss_epoch600")
model.load_state_dict(state)

<All keys matched successfully>

In [ ]:
model.cuda()
model.eval()
test_loader = torch.utils.data.DataLoader(test_data, batch_size=5, shuffle=False)
i = 0

total_mae = 0.0  # total MAE
total_count = 0  # total precossed sample

for imgs, density, counts in iter(test_loader):
    outputs = model(imgs.cuda())
    pred = outputs

    pred_counts = torch.sum(outputs, [1,2])

    mae = torch.abs(pred_counts - counts.cuda()).sum().item()
    total_mae += mae
    total_count += counts.size(0)

    visual(imgs, density, pred, True, 1)
    print(counts)
    print(torch.round(torch.sum(density, [1,2])))
    print(torch.round(torch.sum(pred, [1,2])))

# average mae of total test set
mean_mae = total_mae / total_count
print(f"Mean Absolute Error (MAE): {mean_mae}")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Mean Absolute Error (MAE): 22.051281606739966 -> epoch = 600